In [1]:
import numpy
import process_data

In [2]:
'''
total_to_train = 5
filename = 'data/matrix_700k.txt'
board, move_from, move_to = process_data.process_data(filename, 0, total_to_train) # [0,total_to_train)



board2 = numpy.array([numpy.append(board[i],move_from[i]) for i in range(len(board))])

# Training data is .8 of the data
percent_train = .8
upper = int(total_to_train*percent_train)

# for training move_from NN
board_train = board[0:upper]
board_test  = board[(upper + 1):total_to_train]
move_from_train = move_from[0:upper]
move_from_test  = move_from[(upper + 1):total_to_train]

# for training move_to NN
board2_train = board2[0:upper]
board2_test  = board2[(upper + 1):total_to_train]
move_to_train = move_to[0:upper]
move_to_test  = move_to[(upper + 1):total_to_train]


'''

"\ntotal_to_train = 5\nfilename = 'data/matrix_700k.txt'\nboard, move_from, move_to = process_data.process_data(filename, 0, total_to_train) # [0,total_to_train)\n\n\n\nboard2 = numpy.array([numpy.append(board[i],move_from[i]) for i in range(len(board))])\n\n# Training data is .8 of the data\npercent_train = .8\nupper = int(total_to_train*percent_train)\n\n# for training move_from NN\nboard_train = board[0:upper]\nboard_test  = board[(upper + 1):total_to_train]\nmove_from_train = move_from[0:upper]\nmove_from_test  = move_from[(upper + 1):total_to_train]\n\n# for training move_to NN\nboard2_train = board2[0:upper]\nboard2_test  = board2[(upper + 1):total_to_train]\nmove_to_train = move_to[0:upper]\nmove_to_test  = move_to[(upper + 1):total_to_train]\n\n\n"

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.callbacks

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.batch_counter = 1

    def on_batch_end(self, batch, logs={}):
        if self.batch_counter % 1000 == 0:
            print("LOSS: ",logs.get('loss'),"ACC:",logs.get('acc'))
            self.batch_counter = 1
        self.batch_counter += 1
''' 
From model NN
'''
def get_base_model_from():
        model = Sequential()
        model.add(Dense(288, activation='relu', input_dim=72))
        model.add(Dense(144, activation='relu'))
        #model.add(Dropout(.5))
        model.add(Dense(72, activation='relu'))
        model.add(Dense(72, activation='relu'))
        model.add(Dropout(.5))
        model.add(Dense(72, activation='softmax'))       
        model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']) 
        return model
    
        '''
        # 25% validation 40 epoch small data
        model = Sequential()
        model.add(Dense(144, activation='relu', input_dim=72))
        model.add(Dense(72, activation='relu'))
        model.add(Dense(72, activation='relu'))
        model.add(Dropout(.5))
        model.add(Dense(72, activation='softmax'))       
        model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']) 
        return model
        '''
    
def train_from(num_epoch, b_size, b_per_epoch, history=None):
        model = get_base_model_from()

        # Generator to feed training data
        train_gen = process_data.data_generator('data/small_training.txt', batch_size=b_size, to_from='from')

        # Generator to feed validation data
        valid_gen = process_data.data_generator('data/small_validation.txt', batch_size=100, to_from='from')

        #model.fit_generator(train_gen, epochs=num_epoch, steps_per_epoch=b_per_epoch, validation_data=valid_gen, validation_steps=50, callbacks=[history], verbose=1)
        model.fit_generator(train_gen, epochs=num_epoch, steps_per_epoch=b_per_epoch, validation_data=valid_gen, validation_steps=50, verbose=1)

        return model
    
def test_from(num_epoch):
    model = get_base_model_from()
    model.fit(board_train,move_from_train,epochs=num_epoch, batch_size=500)
    return model.evaluate(board_test, move_from_test, batch_size=500)


'''
To model NN
'''
def get_base_model_to():
        model = Sequential()
        model.add(Dense(288, activation='relu', input_dim=144))
        model.add(Dense(144, activation='relu'))
        model.add(Dropout(.5))
        model.add(Dense(72, activation='softmax'))       
        model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']) 
        return model
    
def train_to(num_epoch,):
        model = get_base_model_to()
        model.fit(board2_train,move_to_train,epochs=num_epoch, batch_size=500)
        return model
    
def test_to(num_epoch):
    model = get_base_model_to()
    model.fit(board2_train,move_to_train,epochs=num_epoch, batch_size=500)
    return model.evaluate(board2_test, move_to_test, batch_size=500)



In [16]:
n_epoch = 200
b_size = 1000
batch_per_epoch = 40
history = LossHistory()
model_from = train_from(n_epoch, b_size, batch_per_epoch, history=None)

Epoch 1/200
40/40 [==============================] - 4s - loss: 4.1556 - acc: 0.0347 - val_loss: 3.8384 - val_acc: 0.0774
Epoch 2/200
40/40 [==============================] - 4s - loss: 3.8753 - acc: 0.0593 - val_loss: 3.6348 - val_acc: 0.1115
Epoch 3/200
40/40 [==============================] - 3s - loss: 3.7461 - acc: 0.0728 - val_loss: 3.5672 - val_acc: 0.1345
Epoch 4/200
40/40 [==============================] - 3s - loss: 3.6563 - acc: 0.0844 - val_loss: 3.4460 - val_acc: 0.1521
Epoch 5/200
40/40 [==============================] - 3s - loss: 3.5911 - acc: 0.1014 - val_loss: 3.3669 - val_acc: 0.1725
Epoch 6/200
40/40 [==============================] - 3s - loss: 3.5512 - acc: 0.1077 - val_loss: 3.3693 - val_acc: 0.1810
Epoch 7/200
40/40 [==============================] - 4s - loss: 3.5058 - acc: 0.1200 - val_loss: 3.3775 - val_acc: 0.1760
Epoch 8/200
40/40 [==============================] - 3s - loss: 3.4810 - acc: 0.1257 - val_loss: 3.3191 - val_acc: 0.1798
Epoch 9/200
40/40 [=====

40/40 [==============================] - 3s - loss: 2.9250 - acc: 0.2414 - val_loss: 2.9854 - val_acc: 0.2608
Epoch 67/200
40/40 [==============================] - 3s - loss: 2.9125 - acc: 0.2439 - val_loss: 3.0137 - val_acc: 0.2436
Epoch 68/200
40/40 [==============================] - 3s - loss: 2.9253 - acc: 0.2434 - val_loss: 2.9637 - val_acc: 0.2467
Epoch 69/200
40/40 [==============================] - 3s - loss: 2.9211 - acc: 0.2391 - val_loss: 2.8912 - val_acc: 0.2671
Epoch 70/200
40/40 [==============================] - 3s - loss: 2.9161 - acc: 0.2456 - val_loss: 2.9924 - val_acc: 0.2628
Epoch 71/200
40/40 [==============================] - 3s - loss: 2.9022 - acc: 0.2480 - val_loss: 3.0217 - val_acc: 0.2394
Epoch 72/200
40/40 [==============================] - 3s - loss: 2.9081 - acc: 0.2442 - val_loss: 2.9630 - val_acc: 0.2459
Epoch 73/200
40/40 [==============================] - 3s - loss: 2.9042 - acc: 0.2442 - val_loss: 2.9129 - val_acc: 0.2681
Epoch 74/200
40/40 [=========

40/40 [==============================] - 3s - loss: 2.7399 - acc: 0.2835 - val_loss: 3.0576 - val_acc: 0.2473
Epoch 132/200
40/40 [==============================] - 3s - loss: 2.7383 - acc: 0.2804 - val_loss: 2.9798 - val_acc: 0.26320.2
Epoch 133/200
40/40 [==============================] - 3s - loss: 2.7417 - acc: 0.2835 - val_loss: 3.0170 - val_acc: 0.2653
Epoch 134/200
40/40 [==============================] - 3s - loss: 2.7505 - acc: 0.2811 - val_loss: 3.1265 - val_acc: 0.2604
Epoch 135/200
40/40 [==============================] - 3s - loss: 2.7371 - acc: 0.2827 - val_loss: 3.0231 - val_acc: 0.2545
Epoch 136/200
40/40 [==============================] - 4s - loss: 2.7395 - acc: 0.2832 - val_loss: 2.9589 - val_acc: 0.2663
Epoch 137/200
40/40 [==============================] - 3s - loss: 2.7230 - acc: 0.2847 - val_loss: 3.0096 - val_acc: 0.2705
Epoch 138/200
40/40 [==============================] - 4s - loss: 2.7248 - acc: 0.2861 - val_loss: 3.1170 - val_acc: 0.2626
Epoch 139/200
40/40

40/40 [==============================] - 4s - loss: 2.6634 - acc: 0.2961 - val_loss: 3.1315 - val_acc: 0.26020.29 - ETA: 1s - loss: 2.6594 -  - ETA: 0s - loss: 2.6627 - acc: 0.
Epoch 190/200
40/40 [==============================] - 4s - loss: 2.6598 - acc: 0.3009 - val_loss: 3.1584 - val_acc: 0.25430.30 - ETA: 0s - loss: 2.6584 - acc: 0.3
Epoch 191/200
40/40 [==============================] - 3s - loss: 2.6534 - acc: 0.3038 - val_loss: 3.1246 - val_acc: 0.2570
Epoch 192/200
40/40 [==============================] - 4s - loss: 2.6434 - acc: 0.3057 - val_loss: 3.0093 - val_acc: 0.2679
Epoch 193/200
40/40 [==============================] - 3s - loss: 2.6534 - acc: 0.3003 - val_loss: 3.0999 - val_acc: 0.2622
Epoch 194/200
40/40 [==============================] - 3s - loss: 2.6578 - acc: 0.3001 - val_loss: 3.1983 - val_acc: 0.2584
Epoch 195/200
40/40 [==============================] - 4s - loss: 2.6480 - acc: 0.3045 - val_loss: 3.1264 - val_acc: 0.2582
Epoch 196/200
40/40 [==================

In [ ]:


model_from.save('engines/from.h5')
print("DONE!")
#model_to.save('engines/to.h5')




'''
test = board_train[0]
test = test.reshape((1,72))

z = model.predict(test)
z = list(z[0])

results = []
for i in range(len(z)):
    results.append((i,z[i]))
    
re = sorted(results, key = lambda x: x[1], reverse=True)

re
'''
#model_to = train_to(20)

In [ ]:

#model_to.save('engines/to.h5')

In [ ]:


#model.save('move_to_model.h5')

In [ ]:
'''
test = board_train[0]
test = test.reshape((1,72))

z = model.predict(test)
z = list(z[0])

results = []
for i in range(len(z)):
    results.append((i,z[i]))
    
re = sorted(results, key = lambda x: x[1], reverse=True)

re
'''